In [ ]:
import pandas as pd
from sklearn.datasets import make_regression
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import os
import seaborn as sns
import pandas as pd
from math import e,sqrt
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
from scipy.optimize import leastsq
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

In [ ]:
!pip3 install -U git+https://github.com/PYFTS/pyFTS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/PYFTS/pyFTS to /tmp/pip-req-build-gsafsq23
  Running command git clone --filter=blob:none --quiet https://github.com/PYFTS/pyFTS /tmp/pip-req-build-gsafsq23
  Resolved https://github.com/PYFTS/pyFTS to commit 6a6eb77d0df983966ae8b29a307e9992c4162a09
  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import datetime
import statistics
import math
import os
import sys
import statistics
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import RandomizedSearchCV

from scipy import stats
from scipy.stats import randint as sp_randint
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

In [ ]:
# convert series to supervised learning
def series_to_supervised_mimo(data, n_in, n_out, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(df.columns[j]+'(t-%d)' % (i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [(df.columns[j]+'(t)') for j in range(n_vars)]
		else:
			names += [(df.columns[j]+'%d(t+%d)' % (j, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
import numpy as np
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame)
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [ ]:
df = pd.read_csv('/content/energydata_complete.csv', sep=',')
df = df.drop(labels=['date','rv1', 'rv2'], axis=1)
df = clean_dataset(df)
df.head()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,60.0,30.0,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3
1,60.0,30.0,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2
2,50.0,30.0,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1
3,50.0,40.0,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0
4,60.0,40.0,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9


In [ ]:
reframed = series_to_supervised_mimo(df, 1, 1)
reframed.head()

,Appliances(t-1),lights(t-1),T1(t-1),RH_1(t-1),T2(t-1),RH_2(t-1),T3(t-1),RH_3(t-1),T4(t-1),RH_4(t-1),...,T8(t),RH_8(t),T9(t),RH_9(t),T_out(t),Press_mm_hg(t),RH_out(t),Windspeed(t),Visibility(t),Tdewpoint(t)
1,60.0,30.0,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2
2,60.0,30.0,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1
3,50.0,30.0,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0
4,50.0,40.0,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9
5,60.0,40.0,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,18.1,48.590000,17.000000,45.29,6.016667,734.0,92.0,5.333333,43.833333,4.8


In [ ]:
reframed.columns

Index(['Appliances(t-1)', 'lights(t-1)', 'T1(t-1)', 'RH_1(t-1)', 'T2(t-1)',
       'RH_2(t-1)', 'T3(t-1)', 'RH_3(t-1)', 'T4(t-1)', 'RH_4(t-1)', 'T5(t-1)',
       'RH_5(t-1)', 'T6(t-1)', 'RH_6(t-1)', 'T7(t-1)', 'RH_7(t-1)', 'T8(t-1)',
       'RH_8(t-1)', 'T9(t-1)', 'RH_9(t-1)', 'T_out(t-1)', 'Press_mm_hg(t-1)',
       'RH_out(t-1)', 'Windspeed(t-1)', 'Visibility(t-1)', 'Tdewpoint(t-1)',
       'Appliances(t)', 'lights(t)', 'T1(t)', 'RH_1(t)', 'T2(t)', 'RH_2(t)',
       'T3(t)', 'RH_3(t)', 'T4(t)', 'RH_4(t)', 'T5(t)', 'RH_5(t)', 'T6(t)',
       'RH_6(t)', 'T7(t)', 'RH_7(t)', 'T8(t)', 'RH_8(t)', 'T9(t)', 'RH_9(t)',
       'T_out(t)', 'Press_mm_hg(t)', 'RH_out(t)', 'Windspeed(t)',
       'Visibility(t)', 'Tdewpoint(t)'],
      dtype='object')

In [ ]:
from sklearn.model_selection import train_test_split
x,y=reframed.loc[:,:],reframed.loc[:,:]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
# Get the number of inputs and outputs from the dataset
df_trainx=x_train.loc[:,'Appliances(t-1)': 'Tdewpoint(t-1)']
df_testx=x_test.loc[:,'Appliances(t-1)': 'Tdewpoint(t-1)']
df_trainy=y_train.loc[:,'Appliances(t)': 'Tdewpoint(t)']
df_original=y_test.loc[:,'Appliances(t)': 'Tdewpoint(t)']



In [ ]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import ExtraTreesRegressor
estimator = ExtraTreesRegressor ()
model = MultiOutputRegressor(estimator)


hyperparameters = dict(estimator__min_samples_split=list(range(2,11)),
                     estimator__max_depth=list(range(2,30)), estimator__min_samples_leaf=list(range(1,11)),
                     estimator__n_estimators=[20,50,100, 200, 300]
                      )

randomized_search = RandomizedSearchCV(model, hyperparameters, random_state=0, n_iter=5, scoring=None,
                                       n_jobs=2, refit=True, cv=3, verbose=True,
                                       pre_dispatch='2*n_jobs', error_score='raise', return_train_score=True)

hyperparameters_tuning = randomized_search.fit(df_trainx, df_trainy)
print('Best Parameters = {}'.format(hyperparameters_tuning.best_params_))

tuned_model = hyperparameters_tuning.best_estimator_

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.8/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best Parameters = {'estimator__n_estimators': 300, 'estimator__min_samples_split': 10, 'estimator__min_samples_leaf': 8, 'estimator__max_depth': 12}


In [ ]:
hyperparameters_tuning.best_params_


{'estimator__n_estimators': 300,
 'estimator__min_samples_split': 10,
 'estimator__min_samples_leaf': 8,
 'estimator__max_depth': 12}

In [ ]:
y_p=tuned_model.predict(df_testx)
columns=df_original.columns
y_p=pd.DataFrame(y_p,columns=columns)
y_p

,Appliances(t),lights(t),T1(t),RH_1(t),T2(t),RH_2(t),T3(t),RH_3(t),T4(t),RH_4(t),...,T8(t),RH_8(t),T9(t),RH_9(t),T_out(t),Press_mm_hg(t),RH_out(t),Windspeed(t),Visibility(t),Tdewpoint(t)
0,48.417256,0.438009,20.876355,35.401267,17.750982,39.154838,20.285484,36.889417,19.763288,34.195200,...,23.052817,39.746367,19.288147,38.997085,1.763664,768.890212,88.432436,2.370297,62.763868,0.007467
1,94.525796,8.205118,21.907035,51.879352,21.304429,45.235651,21.616467,47.435087,20.509107,41.027491,...,20.632544,47.598577,16.295957,44.178921,2.707657,754.588824,89.989960,3.011310,27.077917,1.210815
2,52.478358,1.076487,21.365057,35.504592,17.633494,40.500985,21.655723,35.212410,20.281015,33.759918,...,22.088520,39.608209,19.405339,38.278975,0.196273,766.179456,82.985746,2.002301,64.489343,-2.388433
3,54.846737,1.435045,21.309945,41.278281,23.749775,35.601479,22.046429,36.882442,22.311743,35.298522,...,21.210364,38.170996,20.280998,38.417613,8.811747,767.768094,71.559330,4.992502,39.938698,3.783814
4,74.448737,1.132430,19.948014,35.202829,16.486611,40.112427,20.003253,36.411068,19.255629,34.958331,...,22.091504,39.577573,18.982762,40.653495,-0.448470,769.661528,93.411004,4.026965,47.709696,-1.442729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4929,58.088011,0.461093,24.792827,47.240626,22.693038,49.281303,26.522908,44.416569,25.193199,44.299078,...,25.784267,49.751543,23.999357,47.972754,12.289403,744.762264,98.635546,1.011915,59.430723,12.093537
4930,62.322159,0.744970,20.785779,40.628833,17.607128,46.809232,20.803100,38.281298,20.593903,35.711737,...,22.275266,41.804170,19.509548,39.197939,4.392762,764.317544,87.976029,4.002324,55.594333,2.603637
4931,99.786070,6.458522,21.081668,37.629660,19.899882,37.210739,20.215872,37.984386,22.170843,36.091406,...,21.294444,37.401409,18.605103,41.593344,6.288894,748.391536,76.829586,5.673978,39.935511,2.436800
4932,86.454071,7.816885,23.183556,39.352314,22.387430,38.233991,21.192337,41.888669,21.638040,43.061265,...,20.783108,44.962419,17.546809,44.359278,2.524298,760.409357,95.347125,3.322382,34.458107,1.835772


In [ ]:
col=df_original.columns
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)
result = {

         "rmse": [],
         'nrmse':[],
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]
    }


for col in col:
  original = df_original[col].values
  forecast = y_p[col].values
  mae = round(mean_absolute_error(original,forecast),3)
  r2 = round(r2_score(original,forecast),3)
  rmse = round(mean_squared_error(original,forecast,squared=False),3)
  mape = round(mean_absolute_percentage_error(original,forecast),3)
  nrmse = round(cal_nrmse(rmse, original),3)
  result["rmse"].append(rmse)
  result["mae"].append(mae)
  result["r2"].append(r2)
  result["mape"].append(mape)
  result["nrmse"].append(nrmse)
  result["variable"].append(col)

measures = pd.DataFrame(result)

In [ ]:
import statistics
final_result = {
    "variable": [],
    "rmse": [],
   "nrmse": [],
    "mae": [],
    "r2": [],
    "mape":[]
}


var = measures.groupby("variable")

for col in columns:

    var_agr = var.get_group(col)

    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["mape"].append(mape)

    print(f'Results: {(col,rmse,mae,r2,mape)}')


final_measures_kpca = pd.DataFrame(final_result)

print("Statistics MIMO (test): ")
final_measures_kpca

Results: ('Appliances(t)', 64.53, 28.653, 0.59, 0.272)
Results: ('lights(t)', 4.441, 2.407, 0.673, 4364532062947471.5)
Results: ('T1(t)', 0.045, 0.031, 0.999, 0.001)
Results: ('RH_1(t)', 0.441, 0.174, 0.988, 0.004)
Results: ('T2(t)', 0.095, 0.053, 0.998, 0.003)
Results: ('RH_2(t)', 0.257, 0.138, 0.996, 0.003)
Results: ('T3(t)', 0.065, 0.038, 0.999, 0.002)
Results: ('RH_3(t)', 0.18, 0.092, 0.997, 0.002)
Results: ('T4(t)', 0.074, 0.038, 0.999, 0.002)
Results: ('RH_4(t)', 0.164, 0.096, 0.999, 0.002)
Results: ('T5(t)', 0.129, 0.042, 0.995, 0.002)
Results: ('RH_5(t)', 2.241, 0.586, 0.939, 0.01)
Results: ('T6(t)', 0.236, 0.154, 0.998, 415754773502.058)
Results: ('RH_6(t)', 1.349, 0.69, 0.998, 0.034)
Results: ('T7(t)', 0.04, 0.025, 1.0, 0.001)
Results: ('RH_7(t)', 0.162, 0.095, 0.999, 0.003)
Results: ('T8(t)', 0.048, 0.034, 0.999, 0.002)
Results: ('RH_8(t)', 0.213, 0.146, 0.998, 0.003)
Results: ('T9(t)', 0.046, 0.021, 0.999, 0.001)
Results: ('RH_9(t)', 0.181, 0.11, 0.998, 0.003)
Results: ('T_

,variable,rmse,nrmse,mae,r2,mape
0,Appliances(t),64.530,0.061,28.653,0.590,2.720000e-01
1,lights(t),4.441,0.089,2.407,0.673,4.364532e+15
2,T1(t),0.045,0.005,0.031,0.999,1.000000e-03
3,RH_1(t),0.441,0.014,0.174,0.988,4.000000e-03
4,T2(t),0.095,0.007,0.053,0.998,3.000000e-03
5,RH_2(t),0.257,0.008,0.138,0.996,3.000000e-03
6,T3(t),0.065,0.005,0.038,0.999,2.000000e-03
7,RH_3(t),0.180,0.009,0.092,0.997,2.000000e-03
8,T4(t),0.074,0.007,0.038,0.999,2.000000e-03
9,RH_4(t),0.164,0.007,0.096,0.999,2.000000e-03


#MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scalerx = MinMaxScaler()
scalery=MinMaxScaler()

In [ ]:
from sklearn.model_selection import train_test_split
x,y=reframed.loc[:,:],reframed.loc[:,:]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
# Get the number of inputs and outputs from the dataset



# Get the number of inputs and outputs from the dataset
df_trainx=x_train.loc[:,'Appliances(t-1)': 'Tdewpoint(t-1)']
df_trainx_scaled = scalerx.fit_transform(df_trainx)
df_testx=x_test.loc[:,'Appliances(t-1)': 'Tdewpoint(t-1)']
df_testx_scaled = scalerx.transform(df_testx)
df_trainy=y_train.loc[:,'Appliances(t)': 'Tdewpoint(t)']
df_trainy_scaled=scalery.fit_transform(df_trainy)

df_original=y_test.loc[:,'Appliances(t)': 'Tdewpoint(t)']


In [ ]:

estimator = ExtraTreesRegressor ()
model = MultiOutputRegressor(estimator)


hyperparameters = dict(estimator__min_samples_split=list(range(2,11)),
                     estimator__max_depth=list(range(2,30)), estimator__min_samples_leaf=list(range(1,11)),
                     estimator__n_estimators=[20,50,100, 200, 300,500]
                      )

randomized_search = RandomizedSearchCV(model, hyperparameters, random_state=0, n_iter=5, scoring=None,
                                       n_jobs=2, refit=True, cv=3, verbose=True,
                                       pre_dispatch='2*n_jobs', error_score='raise', return_train_score=True)


hyperparameters_tuning = randomized_search.fit(df_trainx_scaled, df_trainy_scaled)
print('Best Parameters = {}'.format(hyperparameters_tuning.best_params_))

tuned_model = hyperparameters_tuning.best_estimator_

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.8/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best Parameters = {'estimator__n_estimators': 500, 'estimator__min_samples_split': 10, 'estimator__min_samples_leaf': 10, 'estimator__max_depth': 21}


In [ ]:
hyperparameters_tuning.best_params_


{'estimator__n_estimators': 500,
 'estimator__min_samples_split': 10,
 'estimator__min_samples_leaf': 10,
 'estimator__max_depth': 21}

In [ ]:
y_p=tuned_model.predict(df_testx_scaled)
columns=df_original.columns
y_p=pd.DataFrame(y_p,columns=columns)
y_p

,Appliances(t),lights(t),T1(t),RH_1(t),T2(t),RH_2(t),T3(t),RH_3(t),T4(t),RH_4(t),...,T8(t),RH_8(t),T9(t),RH_9(t),T_out(t),Press_mm_hg(t),RH_out(t),Windspeed(t),Visibility(t),Tdewpoint(t)
0,0.035830,0.002050,0.430609,0.229979,0.119702,0.526036,0.256758,0.381217,0.420358,0.254671,...,0.616933,0.347647,0.457800,0.405867,0.217341,0.921078,0.849052,0.175809,0.952042,0.299304
1,0.079151,0.117355,0.541300,0.670266,0.379377,0.695374,0.366914,0.865925,0.489519,0.556778,...,0.394886,0.617059,0.146664,0.621579,0.247684,0.588536,0.866935,0.222641,0.400987,0.354044
2,0.037735,0.024144,0.481867,0.233082,0.111562,0.564679,0.369403,0.302619,0.466409,0.236018,...,0.529138,0.341689,0.470077,0.376968,0.166638,0.858046,0.775655,0.148195,0.976110,0.191229
3,0.043496,0.031567,0.478180,0.390030,0.552841,0.424005,0.403206,0.379786,0.649252,0.304522,...,0.448183,0.293546,0.561474,0.382768,0.444425,0.895009,0.624242,0.369964,0.598967,0.471493
4,0.057676,0.015881,0.333724,0.225073,0.027927,0.553197,0.232541,0.358584,0.373926,0.289094,...,0.529337,0.341356,0.425126,0.476494,0.146462,0.938725,0.913588,0.300776,0.721067,0.234272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4929,0.046141,0.008836,0.844889,0.556358,0.479695,0.810317,0.774353,0.733602,0.909286,0.699759,...,0.867842,0.690348,0.947496,0.778373,0.555962,0.359199,0.983029,0.074649,0.900975,0.849689
4930,0.053072,0.009627,0.421186,0.375410,0.108554,0.740090,0.299092,0.446460,0.495217,0.320858,...,0.545790,0.417940,0.481194,0.415164,0.302311,0.814157,0.840756,0.296558,0.838885,0.417610
4931,0.094609,0.092831,0.453114,0.292161,0.276496,0.470588,0.250463,0.432991,0.637849,0.338632,...,0.457369,0.266984,0.387417,0.514574,0.362729,0.442925,0.697352,0.420485,0.599213,0.410371
4932,0.070679,0.108210,0.674816,0.337758,0.457062,0.499479,0.333675,0.614913,0.589183,0.644653,...,0.409676,0.525770,0.276329,0.629183,0.241913,0.723778,0.939016,0.248162,0.512938,0.383196


In [ ]:
y_p_inv = scalery.inverse_transform(y_p)
columns=df_original.columns
y_p_inv=pd.DataFrame(y_p_inv,columns=columns)
y_p_inv

,Appliances(t),lights(t),T1(t),RH_1(t),T2(t),RH_2(t),T3(t),RH_3(t),T4(t),RH_4(t),...,T8(t),RH_8(t),T9(t),RH_9(t),T_out(t),Press_mm_hg(t),RH_out(t),Windspeed(t),Visibility(t),Tdewpoint(t)
0,48.338046,0.143491,20.867871,35.379989,17.746694,39.170945,20.290334,36.895488,19.765973,34.196582,...,23.045632,39.744328,19.289459,38.972421,1.759320,768.906335,88.527957,2.373426,62.882724,-0.015311
1,94.691588,8.214855,21.916114,51.378577,21.318962,45.193139,21.616178,47.231083,20.533666,41.044051,...,20.620135,47.605794,16.299445,44.184013,2.702979,754.607061,89.887097,3.005649,27.064178,1.188962
2,50.375920,1.690048,21.353278,35.492762,17.634727,40.545183,21.646139,35.219504,20.277145,33.773808,...,22.086616,39.570488,19.407444,38.274220,0.182451,766.195971,82.949788,2.000637,64.447156,-2.392964
3,56.540365,2.209674,21.318361,41.195733,23.705256,35.542367,22.052990,36.864979,22.306692,35.326484,...,21.202317,38.165672,20.285768,38.414334,8.821627,767.785407,71.442377,4.994520,39.932871,3.772848
4,71.713797,1.111637,19.950366,35.201723,16.484180,40.136870,19.998865,36.412863,19.250580,34.976808,...,22.088793,39.560765,18.975463,40.678753,-0.445034,769.665191,93.432701,4.060476,47.869350,-1.446011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4929,59.370643,0.618492,24.791100,47.239512,22.699004,49.280893,26.520114,44.409509,25.193080,44.284815,...,25.786397,49.744351,23.995435,47.972159,12.290434,744.745541,98.710169,1.007761,59.563355,12.093161
4930,66.787569,0.673898,20.778635,40.664484,17.593338,46.783405,20.799867,38.286676,20.596906,35.696760,...,22.268517,41.795493,19.514273,39.197027,4.401883,764.308749,87.897442,4.003533,55.527525,2.587412
4931,111.231876,6.498157,21.080993,37.639477,19.903660,37.199008,20.214569,37.999474,22.180121,36.099630,...,21.302659,37.390591,18.613080,41.598769,6.280863,748.345793,76.998756,5.676541,39.948845,2.428164
4932,85.626597,7.574681,23.180509,39.296332,22.387651,38.226466,21.216114,41.878661,21.639933,43.035810,...,20.781698,44.941957,17.545523,44.367737,2.523489,760.422445,95.365243,3.350187,34.340979,1.830308


In [ ]:
df_original

,Appliances(t),lights(t),T1(t),RH_1(t),T2(t),RH_2(t),T3(t),RH_3(t),T4(t),RH_4(t),...,T8(t),RH_8(t),T9(t),RH_9(t),T_out(t),Press_mm_hg(t),RH_out(t),Windspeed(t),Visibility(t),Tdewpoint(t)
8981,40.0,0.0,20.856667,35.363333,17.70,39.200,20.290000,36.900000,19.700000,34.200000,...,23.043750,40.251250,19.290000,39.090000,1.783333,768.850000,88.166667,2.666667,63.000000,0.000000
2755,90.0,10.0,21.890000,50.890000,21.29,45.060,21.566667,48.524583,20.356667,40.933333,...,20.600000,47.416667,16.290000,44.163333,2.733333,754.550000,89.500000,3.166667,27.333333,1.166667
9133,40.0,0.0,21.390000,35.500000,17.60,40.530,21.600000,35.200000,20.260000,33.700000,...,22.100000,39.700000,19.390000,38.372500,0.083333,766.166667,83.500000,1.833333,64.666667,-2.433333
14359,50.0,0.0,21.390000,41.033333,23.89,34.840,22.033333,36.933333,22.390000,35.236000,...,21.200000,38.090000,20.290000,38.560000,8.850000,767.800000,70.833333,5.166667,40.000000,3.716667
8876,70.0,0.0,19.890000,35.200000,16.50,40.090,20.000000,36.400000,19.230000,35.156667,...,22.100000,39.663333,19.000000,40.700000,-0.333333,769.700000,92.666667,4.000000,50.666667,-1.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17496,60.0,0.0,24.790000,47.090000,22.70,49.290,26.533333,44.500000,25.142857,44.212857,...,25.790000,49.590000,23.968571,47.942857,12.300000,744.700000,99.000000,1.000000,61.000000,12.100000
9283,50.0,0.0,20.790000,40.700000,17.60,46.790,20.790000,38.230000,20.566667,35.700000,...,22.290000,41.760000,19.566667,39.200000,4.216667,764.300000,88.500000,3.833333,55.000000,2.500000
7472,60.0,0.0,21.085714,37.470000,19.89,37.270,20.200000,38.000000,22.075000,35.785000,...,21.264286,37.264286,18.600000,41.700000,6.566667,748.400000,75.333333,5.333333,40.000000,2.433333
611,80.0,10.0,23.200000,39.363333,22.39,38.260,21.133333,41.863333,21.700000,42.900000,...,20.790000,45.096667,17.600000,44.960000,2.616667,760.500000,95.166667,3.166667,37.166667,1.933333


In [ ]:
col=df_original.columns
result = {

         "rmse": [],
         'nrmse':[],
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]
    }


for col in col:
  original = df_original[col].values
  forecast = y_p_inv[col].values
  mae = round(mean_absolute_error(original,forecast),3)
  r2 = round(r2_score(original,forecast),3)
  rmse = round(mean_squared_error(original,forecast,squared=False),3)
  mape = round(mean_absolute_percentage_error(original,forecast),3)

  result["rmse"].append(rmse)
  result["mae"].append(mae)
  result["r2"].append(r2)
  result["mape"].append(mape)
  result["nrmse"].append(nrmse)
  result["variable"].append(col)

measures = pd.DataFrame(result)

In [ ]:
import statistics
final_result = {
    "variable": [],
    "rmse": [],
    "nrmse": [],
    "mae": [],
    "r2": [],
    "mape":[]
}


var = measures.groupby("variable")

for col in columns:

    var_agr = var.get_group(col)

    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["mape"].append(mape)

    print(f'Results: {(col,rmse,mae,r2,mape)}')


final_measures_kpca = pd.DataFrame(final_result)

print("Statistics MIMO (test): ")
final_measures_kpca

Results: ('Appliances(t)', 64.705, 28.971, 0.588, 0.276)
Results: ('lights(t)', 4.423, 2.379, 0.675, 4260427155504943.0)
Results: ('T1(t)', 0.045, 0.032, 0.999, 0.001)
Results: ('RH_1(t)', 0.451, 0.176, 0.987, 0.004)
Results: ('T2(t)', 0.095, 0.053, 0.998, 0.003)
Results: ('RH_2(t)', 0.259, 0.138, 0.996, 0.004)
Results: ('T3(t)', 0.066, 0.039, 0.999, 0.002)
Results: ('RH_3(t)', 0.182, 0.093, 0.997, 0.002)
Results: ('T4(t)', 0.074, 0.039, 0.999, 0.002)
Results: ('RH_4(t)', 0.165, 0.097, 0.999, 0.003)
Results: ('T5(t)', 0.129, 0.043, 0.995, 0.002)
Results: ('RH_5(t)', 2.246, 0.589, 0.939, 0.01)
Results: ('T6(t)', 0.235, 0.152, 0.998, 430384532943.759)
Results: ('RH_6(t)', 1.349, 0.688, 0.998, 0.034)
Results: ('T7(t)', 0.041, 0.026, 1.0, 0.001)
Results: ('RH_7(t)', 0.161, 0.094, 0.999, 0.003)
Results: ('T8(t)', 0.047, 0.035, 0.999, 0.002)
Results: ('RH_8(t)', 0.212, 0.145, 0.998, 0.003)
Results: ('T9(t)', 0.046, 0.021, 0.999, 0.001)
Results: ('RH_9(t)', 0.181, 0.108, 0.998, 0.003)
Results

,variable,rmse,nrmse,mae,r2,mape
0,Appliances(t),64.705,0.092,28.971,0.588,2.760000e-01
1,lights(t),4.423,0.092,2.379,0.675,4.260427e+15
2,T1(t),0.045,0.092,0.032,0.999,1.000000e-03
3,RH_1(t),0.451,0.092,0.176,0.987,4.000000e-03
4,T2(t),0.095,0.092,0.053,0.998,3.000000e-03
5,RH_2(t),0.259,0.092,0.138,0.996,4.000000e-03
6,T3(t),0.066,0.092,0.039,0.999,2.000000e-03
7,RH_3(t),0.182,0.092,0.093,0.997,2.000000e-03
8,T4(t),0.074,0.092,0.039,0.999,2.000000e-03
9,RH_4(t),0.165,0.092,0.097,0.999,3.000000e-03


#Sliding window cross validation

In [ ]:
def create_and_fit_model(X_train,y_train):

    clf = MultiOutputRegressor(estimator=ExtraTreesRegressor(max_depth=12,
                                                     min_samples_leaf=8,min_samples_split=10,
                                                     n_estimators=300))
    clf.fit(X_train, y_train)

    return clf

In [ ]:
def forecast_model(clf, X_test):
    forecast = clf.predict(X_test)
    return forecast

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import math
from pyFTS.common import Util
from pyFTS.benchmarks import Measures


result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }

final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
data=reframed
n_windows = 30
train_size = 0.75
tam = len(data)
windows_length = math.floor(tam / n_windows)
for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:

            print('-' * 20)
            print(f'training window {(ct)}')

            scaler = StandardScaler()


            Xtrain = ttrain.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
            ytrain = ttrain.loc[:,'Appliances(t)':'Tdewpoint(t)']
            Xtest = ttest.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
            ytest = ttest.loc[:,'Appliances(t)':'Tdewpoint(t)']

            # Xtrain = scaler.fit_transform(ttrain.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)'])
            # ytrain = scaler.fit_transform(ttrain.loc[:,'Appliances(t)':'Tdewpoint(t)'])
            # Xtest = scaler.fit_transform(ttest.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)'])
            # ytest = scaler.fit_transform(ttest.loc[:,'Appliances(t)':'Tdewpoint(t)'])


            model = create_and_fit_model(Xtrain,ytrain)
            forecast = forecast_model(model, Xtest)

            df_forecats_columns = ttest.loc[:,'Appliances(t)':'Tdewpoint(t)'].columns

            columns = list(df_forecats_columns)

            # prediction = scaler.inverse_transform(forecast)
            prediction = forecast
            df_forecast = pd.DataFrame(prediction,columns=columns)

            #ytest_metric = scaler.inverse_transform(y_test.reshape(y_test.shape[0], y_test.shape[2]))
            ytest_metric = ttest.loc[:,'Appliances(t)':'Tdewpoint(t)'].values
            df_original = pd.DataFrame(ytest_metric,columns=columns)

            for col in columns:
                original = df_original[col].values
                forecast = df_forecast[col].values
                # print(original)
                # print(forecast)
                mae = round(mean_absolute_error(original,forecast),3)
                r2 = round(r2_score(original,forecast),3)
                rmse = mean_squared_error(original,forecast,squared=False)
                # rmse = round(Measures.rmse(original,forecast),3)
                mape = round(Measures.mape(original,forecast),3)
                nrmse = round(cal_nrmse(rmse, original),3)
                smape = round(Measures.smape(original,forecast),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["smape"].append(smape)
                result["window"].append(ct)
                result["variable"].append(col)

#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()

measures = pd.DataFrame(result)



--------------------
training window 0


/usr/local/lib/python3.8/dist-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/usr/local/lib/python3.8/dist-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 657


/usr/local/lib/python3.8/dist-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/usr/local/lib/python3.8/dist-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/usr/local/lib/python3.8/dist-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 1314


...

/usr/local/lib/python3.8/dist-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 18396


/usr/local/lib/python3.8/dist-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 19053


/usr/local/lib/python3.8/dist-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


In [ ]:
measures.to_csv (r'window_MIMO_extratree.csv', index = False, header=True)

In [ ]:
import statistics
df_forecats_columns = reframed.loc[:,'Appliances(t)':'Tdewpoint(t)'].columns

columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": []
}

var = measures.groupby("variable")

for col in columns:

    var_agr = var.get_group(col)

    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)

    #print(f'Results: {(col,rmse,mae,r2)}')


final_measures = round(pd.DataFrame(final_result),3)

In [ ]:
final_measures

,variable,rmse,mae,mape,r2,smape,nrmse
0,Appliances(t),72.073,43.662,50.892,-5.900000e-02,18.511,0.156
1,lights(t),5.160,3.472,inf,1.360000e-01,84.291,inf
2,T1(t),0.282,0.218,1.034,6.210000e-01,0.516,0.143
3,RH_1(t),1.036,0.718,1.788,4.800000e-01,0.891,0.131
4,T2(t),0.371,0.275,1.350,8.110000e-01,0.675,0.105
5,RH_2(t),0.945,0.711,1.781,5.930000e-01,0.887,0.125
6,T3(t),0.453,0.358,1.560,-4.770000e-01,0.791,0.237
7,RH_3(t),0.689,0.509,1.294,4.950000e-01,0.642,0.147
8,T4(t),0.458,0.366,1.848,-1.560000e-01,0.926,0.231
9,RH_4(t),0.810,0.601,1.515,3.020000e-01,0.758,0.167


In [ ]:
final_measures.to_csv(r"extra_tree.csv",index = False, header=True)